# Ovde ide dobijeni beat

In [206]:
tempo = 100

In [207]:
# Primer beat-a
beats = [
    [1/4, 1/8, 1/8, 1/4, 1/8, 1/8, 1/4, 1/8, 1/8, 1/2],
    [1/4, 1/8, 1/8, 1/4, 1/4, 3/8, 1/8, 1/2],
    [1/2, 1/2, 1/4, 1/8, 1/8, 1/4, 1/8, 3/8]
]
# Ovde ce biti sacuvana izabrana lestvica
scale = []

# Potrebna muzicka teorija

In [208]:
def note_duration(note_length):
    bar_duration = 60 / tempo * 4
    return note_length * bar_duration

In [209]:
# Keys
#              0    1     2    3     4    5     6    7    8     9    10    11
piano_keys = ['c', 'cs', 'd', 'ds', 'e', 'f', 'fs', 'g', 'gs', 'a', 'as', 'b']

# Modes
modes = {
    "major" : [0, 2, 4, 5, 7, 9, 11],
    "minor" : [0, 2, 3, 5, 7, 8, 10]
}

In [210]:
# Note u skali
def scale(root, mode):
    scl = []
    n = piano_keys.index(root)

    for i in modes[mode]:
        scl.append(piano_keys[(i+n)%12])

    return scl

In [211]:
for mode in modes:
    for key in piano_keys:
        print(key + " " + mode + ": ", scale(key, mode))

c major:  ['c', 'd', 'e', 'f', 'g', 'a', 'b']
cs major:  ['cs', 'ds', 'f', 'fs', 'gs', 'as', 'c']
d major:  ['d', 'e', 'fs', 'g', 'a', 'b', 'cs']
ds major:  ['ds', 'f', 'g', 'gs', 'as', 'c', 'd']
e major:  ['e', 'fs', 'gs', 'a', 'b', 'cs', 'ds']
f major:  ['f', 'g', 'a', 'as', 'c', 'd', 'e']
fs major:  ['fs', 'gs', 'as', 'b', 'cs', 'ds', 'f']
g major:  ['g', 'a', 'b', 'c', 'd', 'e', 'fs']
gs major:  ['gs', 'as', 'c', 'cs', 'ds', 'f', 'g']
a major:  ['a', 'b', 'cs', 'd', 'e', 'fs', 'gs']
as major:  ['as', 'c', 'd', 'ds', 'f', 'g', 'a']
b major:  ['b', 'cs', 'ds', 'e', 'fs', 'gs', 'as']
c minor:  ['c', 'd', 'ds', 'f', 'g', 'gs', 'as']
cs minor:  ['cs', 'ds', 'e', 'fs', 'gs', 'a', 'b']
d minor:  ['d', 'e', 'f', 'g', 'a', 'as', 'c']
ds minor:  ['ds', 'f', 'fs', 'gs', 'as', 'b', 'cs']
e minor:  ['e', 'fs', 'g', 'a', 'b', 'c', 'd']
f minor:  ['f', 'g', 'gs', 'as', 'c', 'cs', 'ds']
fs minor:  ['fs', 'gs', 'a', 'b', 'cs', 'd', 'e']
g minor:  ['g', 'a', 'as', 'c', 'd', 'ds', 'f']
gs minor:  ['g

# Sviranje dobijene melodije

In [212]:
import pygame as pg
import time
from threading import Thread

In [213]:
pg.mixer.init()
pg.init()

(5, 0)

In [214]:
def path(note):
    return "Piano/" + note + ".wav"

In [215]:
def play_note(note, duration):
    pg.mixer.music.load(path(note))
    pg.mixer.music.play()
    time.sleep(note_duration(duration))

# Klasa Melody

In [216]:
import random

In [217]:
class Melody:

    def __init__(self, beats):
        self.notes = self.generate_melody(random.choice(beats))
        self.fitness = self.calc_fitness()

    def generate_melody(self, beat):
        notes = []
        for i in range(len(beat)):
            octave = random.randint(4,5)
            note = note = random.choice(scale)
            note = note + str(octave)
            notes.append([note, beat[i]])
        return notes

    def print(self):
        for note in self.notes:
            print(note[0], end=" ")
        print()

    def play(self):
        print("Melody", end=" ")
        self.print()
        pg.mixer.set_num_channels(len(self.notes))
        th = {}
        n = 0
        for i in self.notes:
            th[n] = Thread(target=play_note, args=(i[0], i[1]))
            th[n].start()
            th[n].join()
            n += 1

    def calc_fitness(self):
        self.play()
        rating = int(input("Rating: "))
        print()
        return rating

# Genetic Algorithm

In [218]:
def selection(population, tournament_size):
    chosen = random.sample(population, tournament_size)
    return max(chosen, key=lambda x: x.fitness)

In [219]:
def crossover(parent1, parent2, child1, child2):
    
    random_pos = random.randrange(0, len(parent1.notes))
    
    child1.notes[:random_pos] = parent1.notes[:random_pos]
    child1.notes[random_pos:] = parent2.notes[random_pos:]
    
    child2.notes[:random_pos] = parent2.notes[:random_pos]
    child2.notes[random_pos:] = parent1.notes[random_pos:]

In [220]:
def mutation(melody, mutation_prob):
    for i in range(len(melody.notes)):
        prob = random.random()
        if prob < mutation_prob:
            new_note = random.choice(scale)
            octave = random.randint(4,5)
            melody.notes[i][0] = new_note+str(octave)

In [221]:
def genetic_algorithm(
    num_bars=8, 
    scale_root="c", 
    scale_mode="major", 
    population_size=10,
    elitism_size=2, 
    tournament_size=2, 
    mutation_probability=0.5
):

    genId = 1
    print("GENERATION " + str(genId))
    print()
    population = [Melody(beats) for _ in range(population_size)]
    new_population = population.copy()

    while True:

        if input("Next generation? [y/n]: ") == 'n':
            break

        genId += 1
        print("GENERATION " + str(genId))
        print()
        
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        
        for j in range(elitism_size, population_size, 2):
            
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, child1=new_population[j], child2=new_population[j+1])
    
            mutation(new_population[j], mutation_probability)
            mutation(new_population[j+1], mutation_probability)
            
            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()
    
        population = new_population.copy()
        
    population.sort(key=lambda x: x.fitness, reverse=True)
    return population[:8]

# Main

In [222]:
scale_root = input("Scale root [ex. c for C, cs for C#]: ")
scale_mode = input("Scale mode [ex. major, minor]: ")
scale = scale(scale_root, scale_mode)

Scale root [ex. c for C, cs for C#]:  fs
Scale mode [ex. major, minor]:  major


In [223]:
melodies = genetic_algorithm()

GENERATION 1

Melody f4 gs4 gs4 ds5 gs4 fs5 ds4 b5 b5 cs5 


Rating:  6



Melody fs5 as5 f4 cs4 b4 cs5 ds5 f5 


Rating:  6



Melody fs4 b4 cs5 gs4 cs5 cs4 b5 gs5 fs5 f4 


...


Melody b4 cs4 ds4 b5 f5 cs4 gs5 gs4 


Rating:  9



Melody b5 fs4 ds4 b5 f5 cs4 gs5 gs4 


Rating:  8


Next generation? [y/n]:  n


In [225]:
for melody in melodies:
    melody.play()

Melody fs4 cs4 b5 b5 cs4 as4 fs5 fs4 
Melody fs4 cs4 b5 b5 cs4 as4 cs5 fs4 
Melody b4 cs4 gs4 fs5 cs4 ds4 b5 fs5 
Melody b5 cs5 cs4 as5 gs4 ds4 cs5 fs5 
Melody fs4 cs4 b5 b5 cs4 as4 cs5 fs5 
Melody b5 cs5 cs4 as5 gs4 ds4 cs5 fs5 
Melody b4 cs4 ds4 b5 f5 cs4 gs5 gs4 
Melody b5 fs4 ds4 b5 f5 cs4 gs5 gs4 
